## Running the code (part a)
# SSH To terminal
If you are running your database remotely and only have access to the terminal. You are able to recreate the experience of running the script from the terminal via the following code. The key components to change are:
1. The ssh user name.
2. The path/to/repo.
3. Input.
    - Sample input: "GetStatCumulative\nUS\n2\n2022\n3\n2023"
    - Note that the sample input feeds in all input parameters at once.

This way is not recommended, as you are only able to run one query at a time. If you wish to utilize terminal commands, it would be more convenient to run report_generation.py

In [ ]:
%%bash
ssh gkang9@ugradx.cs.jhu.edu
cd path/to/repo/
python -m venv virenv
source virenv/bin/activate
pip install matplotlib
pip install pymysql
pip install plotly
pip install pandas
pip install termplotlib
echo -e "input" | python /report_generation.py 


### Running the code (part b)
#### Python notebook
If you have access to your database on a local server, or via a non terminal output, you may run the following sections to access the database. This way is recommended if you would like to utilize the graphics demonstrated in the stored procedures: GetTwitchStats and GetStatCumulative.

# Initialize import statements

In [15]:
import os
import pymysql
import pandas as pd
import matplotlib.pyplot as plt

try:
   import plotly.express as px
except:
   print("ERROR: Failed to install plotly (necessary for sunburst plots")
   

# Initialize Helper Methods
The following code block contains helper methods, mostly regarding the output of results. The code documented here is as follows:
- printTable(result):
    - @param result. A dictionary object returned by the sql cursor query
    - credited to StackOverflow user Le Droid.
    - Prints out query results in table format
    - Automatically determines row sizes and column headers
- plotCumulative(result):
    - @param result. A dictionary object returned by the sql cursor query
    - Plots all TwitchStats and covid data over time
- plotSunburst(result):
    - @param result. A dictionary object returned by the sql cursor query
    - Plots the specified twitchstat as an interactive sunburst plot.
    - Layers are formatted as year --> month --> stat value
- plotCumulativeSunburst(result):
    - @param result. A dictionary object returned by the sql cursor query
    - Plots the all stats and covid data as an interactive sunburst plot.
    - Layers are formatted as year --> month --> day --> stat value
- printMethods(result):
    - prints all available functions that you can do.

In [16]:
# Initialize helper methods
# Print in table format
def printTable(result, colList=None):
    # https://stackoverflow.com/questions/17330139/python-printing-a-dictionary-as-a-horizontal-table-with-headers
   colList = list(result[0].keys() if result else [])
   temp = [colList] # 1st row = header
   for item in result:
       temp.append([str(item[col] if item[col] is not None else '') for col in colList])
   colSize = [max(map(len,col)) for col in zip(*temp)]
   formatStr = ' | '.join(["{{:<{}}}".format(i) for i in colSize])
   temp.insert(1, ['-' * i for i in colSize]) # Seperating line
   for item in temp:
       print(formatStr.format(*item))

# Plot results
def plotCumulative(result):
   plt.figure
   colList = list(result.keys())
   print(1, len(colList))
   for idx in range(1, len(colList)):
      if (colList[idx] != "dateofdata" and colList[idx] != "yearNum" and colList[idx] != "monthNum" and colList[idx] != "year" and colList[idx] != "month"):
         plt.plot(result[colList[0]], result[colList[idx]])
   plt.legend(colList[1:(len(colList))])
   plt.show()

# Plot sunburst results
def plotSunburst(result):
   plt.figure
   fig = px.sunburst(result, path['year', 'month'], values = 'selected')
   fig.show()

# Plot multiple sunburst results
def plotCumulativeSunburst(result):

   colList = list(result.keys())
   for idx in range(1, len(colList)):
      if (colList[idx] != "dateofdata" and colList[idx] != "yearNum" and colList[idx] != "monthNum" and colList[idx] != "year" and colList[idx] != "month"):
         plt.figure
         fig = px.sunburst(result, path['yearNum', 'monthNum', 'dateofdata'], values = result[colList[idx]], title=colList[idx])
         fig.show()

# Print available methods
def printMethods():
    print("Use \'q\' or \'quit\' to quit.")
    print("- GetMaxCovid(country VARCHAR(100)) takes in a country name and returns the max number of cases and what month/year they occured for that country.")
    print("- GetAverageRatings(year INT) takes in a year and returns the average rating for film media released in that year.")
    print("- GetCovidPlatforms(year INT, month INT) takes in month/year and returns the average Metacritic rating for games relased in that month.")
    print("- GetStatCumulative(country VARCHAR(100), startMonth INT, startYear INT, stopMonth INT, stopYear INT) takes in a time period (specified by start/stop month/year and a country, plotting COVID cases and twitch stats overtime. The graph display is not interactable with the python script, but is interactable with ipynb")
    print("- GetTwitchStats(command VARCHAR(15)) takes in a twitchStatistcs and returns the specified twitch statistics over available time.\nPossible commands: hoursWatched, avgViewers, peakViewers, avgChannels, peakChannels, hoursStreamed, gamesStreamed, activeAffiliate, activePartners")
    

## Connecting to database
Fill out the following code block with info on how to connect to your database. Sample input has been shown below

In [ ]:
# Initialize database and cursors
db = pymysql.connect(
    host = "dbase.cs.jhu.edu",
    user = "22fa_gkang9",
    password = "password",
    database = "22fa_gkang9_db")
cursor = db.cursor(pymysql.cursors.DictCursor)

## Run method
Once the program beings running, you may type in stored procedure commands in order to access the database. The commands are not case sensitive. 

Enter 'q' or 'quit' to stop the program and close the database connection. 

In [ ]:


# Call methods
try :
    print("Use \'q\' or \'quit\' to quit. Use \'h\' or \'help\' for info on methods")
    method_name = input("Enter method name: ")
    method_name = method_name.lower()
    while (not (method_name == "q" or method_name == "quit")):
        if (method_name == "getmaxcovid"):
            country = input("Enter country name: ")
            cursor.execute("CALL GetMaxCovid(%s)", country)
            results = cursor.fetchall()
            printTable(results)

        elif (method_name == "getaverageratings"):
            year = input("Enter desired year: ")
            cursor.execute("CALL GetAverageRatings(%s)", int(year))
            results = cursor.fetchall()
            printTable(results)

        elif (method_name == "getcovidplatforms"):
            year = input("Enter desired year: ")
            month = input("Enter desired month: ")
            cursor.execute("CALL GetCovidPlatforms(%s, %s)", [year, month])
            results = cursor.fetchall()
            printTable(results)

        elif (method_name == "getstatcumulative"):
            country = input("Enter country name: ")
            startMonth = input("Enter start month: ")
            startYear = input("Enter start year: ")
            stopMonth = input("Enter stop month: ")
            stopYear = input("Enter stop year: ")
            cursor.execute("CALL GetStatCumulative(%s, %s, %s, %s, %s)", [country, startMonth, startYear, stopMonth, stopYear])
            results = cursor.fetchall()
            # Display Results
            printTable(results)
            try:
               plotCumulative(results)
               plotCumulativeSunburst(results)
            except:
               print("Failed to plot results of GetStatCumulative")

        elif (method_name == "getcovidtwitchstats"):
            command = input("Enter desired command: ")
            cursor.execute("CALL GetTwitchStats(%s)", command)
            results = cursor.fetchall()
            print(results)
            printTable(results)
            try:
               plotSunburst(results)
            except:
               print("Failed to plotresults of GetTwitchStats")

        elif (method_name == "help" or method_name == "h"):
            printMethods()

        else:
            print("ERROR: Invalid Command")
        method_name = input("Enter procedure name: ")
except:
    db.commit()
    cursor.close()
    db.close()

try:
    db.commit()
    cursor.close()
    db.close()
    print("Closed connections and executed all queries successfully")
except:
    print("Error occured in querying, connection already closed")